In [118]:
import torch
import numpy as np
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(23644, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [167]:
import torch
import torch.nn.functional as F
from torch.nn import Parameter
from torch_scatter import scatter_mean
from torch_geometric.utils import remove_self_loops, add_self_loops

from torch_geometric.nn.inits import uniform


class SAGEConv(torch.nn.Module):
    r"""The GraphSAGE operator from the `"Inductive Representation Learning on
    Large Graphs" <https://arxiv.org/abs/1706.02216>`_ paper

    .. math::
        \mathbf{\hat{x}}_i &= \mathbf{\Theta} \cdot
        \mathrm{mean}_{j \in \mathcal{N(i) \cup \{ i \}}}(\mathbf{x}_j)

        \mathbf{x}^{\prime}_i &= \frac{\mathbf{\hat{x}}_i}
        {\| \mathbf{\hat{x}}_i \|_2}.

    Args:
        in_channels (int): Size of each input sample.
        out_channels (int): Size of each output sample.
        normalize (bool, optional): If set to :obj:`False`, output features
            will not be :math:`\ell_2`-normalized.
        bias (bool, optional): If set to :obj:`False`, the layer will not learn
            an additive bias. (default: :obj:`True`)
    """

    def __init__(self, in_channels, out_channels, normalize=True, bias=True, features=True):
        super(SAGEConv, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.normalize = normalize
        if features:
            self.weight = Parameter(torch.Tensor(46, self.in_channels, out_channels))
        else:
            self.weight = Parameter(torch.Tensor(self.in_channels, out_channels))

        if bias:
            self.bias = Parameter(torch.Tensor(46,out_channels))
        else:
            self.register_parameter('bias', None)

        self.reset_parameters()

    def reset_parameters(self):
        size = self.weight.size(0)
        uniform(size, self.weight)
        uniform(size, self.bias)


    def forward(self, x, edge_index, edge_type):
        """"""
#         edge_index, _ = remove_self_loops(edge_index)
#         edge_index = add_self_loops(edge_index, num_nodes=x.size(0))
        
        if x is None:
            return self.weight
              
        row, col = edge_index
        pre_feats = x[col]
        new_feats = []
        if self.bias is not None:
            for idx, msg in enumerate(pre_feats):
                ids = edge_type[idx].long()
                new_feats.append(torch.matmul(msg, self.weight[ids]) + self.bias[ids])
        else:
            for idx, msg in enumerate(pre_feats):   
                ids = edge_type[idx].long()
                new_feats.append(torch.matmul(msg, self.weight[ids]) + self.bias[ids])
          
        new_feats = torch.stack(new_feats)
        out = scatter_mean(new_feats[col], row, dim=0, dim_size=x.size(0))
        if self.normalize:
            out = F.normalize(out, p=2, dim=-1)

        return out


    def __repr__(self):
        return '{}({}, {})'.format(self.__class__.__name__, self.in_channels,
                                   self.out_channels)

In [168]:
import torch
import numpy as np
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = SAGEConv(23644, 16, features=False)
        self.conv2 = SAGEConv(16, dataset.num_classes)

    def forward(self, x, data):
        edge_index, edge_type = data.edge_index, data.edge_type
        x = self.conv1(x, edge_index, edge_type)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, edge_type)
        return F.log_softmax(x, dim=1)

In [169]:
from torch_geometric.datasets import Entities
dataset = Entities(root='tmp/',name='MUTAG')

In [170]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = datasets[0].to(device)

In [171]:

model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(None, data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    print(loss)
    loss.backward()
    optimizer.step()

AttributeError: 'Data' object has no attribute 'train_mask'

In [11]:
model.eval()
_, pred = model(data).max(dim=1)
correct = pred[data.test_mask].eq(data.y[data.test_mask]).sum().item()
acc = correct / data.test_mask.sum().item()
print('Accuracy: {:.4f}'.format(acc))

Accuracy: 0.8060


In [77]:
a = np.array([1,2,3])
b = np.array([2,3,4])

In [78]:
a = torch.from_numpy(a)
b = torch.from_numpy(b)
lst = []
lst.append(a)
lst.append(b)

In [117]:
datasets[0].edge_type

tensor([2, 0, 2,  ..., 6, 9, 3])

In [86]:
c = torch.stack(lst)

In [90]:
c

tensor([[1, 2, 3],
        [2, 3, 4]])

In [108]:
c[[0,0,1]]

tensor([[1, 2, 3],
        [1, 2, 3],
        [2, 3, 4]])

In [137]:
dataset = Entities(root='AIFB/',name='AIFB')

Extracting AIFB/aifb.tgz
Processing...
Done!


In [140]:
dataset[0]

Data(edge_index=[2, 58086], edge_norm=[58086], edge_type=[58086], test_idx=[36], test_y=[36], train_idx=[140], train_y=[140])

In [102]:
dataset.num_features

1433

In [106]:
dataset[0].x[[1,1,2]]

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [110]:
x = torch.from_numpy(np.array([]))
x = x.unsqueeze(-1) if x.dim() == 1 else x

In [111]:
x

tensor([], size=(0, 1), dtype=torch.float64)

In [112]:
x = torch.from_numpy(np.array([]))

In [115]:
x.dim()

1

In [141]:
c

tensor([[1, 2, 3],
        [2, 3, 4]])

Parameter containing:
tensor([[[ 4.0490e-32,  4.5864e-41, -3.1699e+20,  3.0851e-41,  0.0000e+00],
         [ 0.0000e+00, -3.7223e+24,  4.5863e-41,  0.0000e+00,  0.0000e+00],
         [-7.7637e-13,  4.5863e-41,  0.0000e+00,  0.0000e+00,  1.4013e-45],
         [ 0.0000e+00,  2.8026e-45,  0.0000e+00,  4.0489e-32,  4.5864e-41]],

        [[-1.7379e+14,  4.5863e-41,  3.3771e-43,  0.0000e+00,  6.1751e-31],
         [ 3.0852e-41,  5.9880e-31,  3.0852e-41,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00, -6.2728e+19,  3.0851e-41, -6.2728e+19],
         [ 3.0851e-41, -6.2728e+19,  3.0851e-41,  2.8026e-45,  0.0000e+00]],

        [[ 2.8026e-45,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00, -6.2728e+19,  3.0851e-41],
         [-6.2728e+19,  3.0851e-41, -6.2728e+19,  3.0851e-41,  1.4013e-45],
         [ 0.0000e+00,  1.4013e-45,  0.0000e+00,  0.0000e+00,  0.0000e+00]],

        [[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00

In [185]:

#uniform(size, self.weight)
size= weight.size(0)
uniform(size, weight)
weight


dataset[0].edge_index



tensor([[    0,     0,     0,  ..., 23643, 23643, 23643],
        [ 2183, 14286, 14888,  ..., 14752, 19951, 22027]])

In [187]:
col, row = dataset[0].edge_index

In [195]:
node = col.numpy()
ids = -1
lst = []
for idx, v in enumerate(node):
    if ids != v:
        ids = v
        lst.append(idx)
        
print(lst)

class dataloader(object):
    def __init__(self, dataset):
        self.dataset = dataset
        self.index = self.process()
    
    
    def process(self):
        col, row = self.dataset[0].edge_index
        node = col.numpy()
        ids = -1
        lst = []
        for idx, v in enumerate(node):
            if ids != v:
                ids = v
                lst.append(idx)
        return np.array(lst)
    
    def sampling(self, depth, num_neighb, nodes, batch):
        if depth == 0:
            return
        for node in nodes:
            begin_idx, end_idx = index[node, node+1]
            tol_neighb = begin_idx - end_idx
            #tol_sample_neighb = min(tol_neighb, m)
            sampled_neighb = np.random.randint(begin_idx, end_idx, size = m)
            batch.append(self.edge_index[sampled_neighb])
            self.sample(depth-1, num_neighb, sampled_neighb)
        
        
    

[0, 4, 10, 14, 16, 20, 24, 28, 32, 36, 40, 44, 48, 52, 56, 60, 64, 68, 72, 76, 80, 82, 84, 88, 92, 96, 98, 105, 107, 113, 117, 121, 126, 130, 134, 138, 140, 146, 152, 159, 163, 169, 173, 175, 179, 181, 185, 187, 191, 195, 199, 203, 207, 209, 213, 219, 223, 227, 231, 235, 239, 243, 247, 251, 255, 261, 268, 280, 284, 732, 736, 740, 755, 762, 764, 768, 772, 776, 782, 786, 790, 794, 798, 803, 807, 811, 815, 817, 821, 825, 829, 833, 837, 841, 843, 847, 854, 858, 864, 868, 872, 876, 883, 886, 890, 894, 898, 901, 905, 911, 915, 919, 925, 929, 931, 937, 940, 944, 950, 956, 960, 964, 968, 974, 980, 982, 984, 988, 992, 996, 1000, 1004, 1008, 1010, 1014, 1018, 1022, 1026, 1030, 1034, 1087, 1094, 1098, 1102, 1106, 1108, 1112, 1116, 1123, 1127, 1129, 1135, 1139, 1143, 1147, 1151, 1153, 1157, 1161, 1165, 1169, 1173, 1175, 1177, 1184, 1188, 1190, 1194, 1198, 1200, 1202, 1206, 1210, 1214, 1218, 1222, 1226, 1230, 1232, 1236, 1238, 1244, 1252, 1256, 1260, 1264, 1266, 1270, 1276, 1280, 1284, 1290, 1291, 

In [210]:
a = datasets[0]
a.num_nodes

23644

In [338]:
from torch.utils.data import Dataset, DataLoader
class MydataSet(Dataset):
    def __init__(self, dataset, depth, num_neighb, transform=None): 
        self.dataset = dataset
        self.depth = depth
        self.num_neighb = num_neighb
        self.index = self.process()
        self.transform = transform
    
    def process(self):
        col, row = self.dataset[0].edge_index
        node = col.numpy()
        ids = -1
        lst = []
        for idx, v in enumerate(node):
            if ids != v:
                ids = v
                lst.append(idx)
        return np.array(lst)
    
    def sampling(self, depth, num_neighb, nodes):
        batch =[]
        mapping = []
        for i in range(depth):
            ends = []
            print(nodes)
            for node in nodes:
                begin_idx = self.index[node]
                end_idx = self.index[node+1]
                tol_neighb = begin_idx - end_idx
                sampled_neighb = np.random.randint(begin_idx, end_idx, size = num_neighb)
                edges = self.dataset[0].edge_index[:, sampled_neighb]
#                 v = 
                _, end_points = edges
                ends.append(end_points)
                batch.append(edges)
            nodes = torch.cat(ends)
        
            
    
    def __len__(self):
        return datasets[0].num_nodes

    def __getitem__(self, idx):
        lst = []
        self.sampling(self.depth, self.num_neighb, [idx], lst)
        sample = torch.cat(lst, dim=1).long()
        if self.transform:
            sample = self.transform(sample)
        return sample, self.dataset[0]
    
    
import torch
import torch.nn.functional as F
from torch.nn import Parameter
from torch_scatter import scatter_mean
from torch_geometric.utils import remove_self_loops, add_self_loops

from torch_geometric.nn.inits import uniform


class MyGCN(torch.nn.Module):

    def __init__(self, in_channels, out_channels, normalize=True, bias=True, number_relation=):
        super(SAGEConv, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.normalize = normalize
        self.weight = Parameter(torch.Tensor(self.in_channels, out_channels))

        if bias:
            self.bias = Parameter(torch.Tensor(out_channels))
        else:
            self.register_parameter('bias', None)

        self.reset_parameters()

    def reset_parameters(self):
        size = self.weight.size(0)
        uniform(size, self.weight)
        uniform(size, self.bias)


    def forward(self, x, edge_index):
        """"""
        x = x.unsqueeze(-1) if x.dim() == 1 else x
        row, col = edge_index

        x = torch.matmul(x, self.weight)
        out = scatter_mean(x[col], row, dim=0, dim_size=x.size(0))

        if self.bias is not None:
            out = out + self.bias

        if self.normalize:
            out = F.normalize(out, p=2, dim=-1)

        return out


    def __repr__(self):
        return '{}({}, {})'.format(self.__class__.__name__, self.in_channels,
                                   self.out_channels)

In [356]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')
G_data = MydataSet(dataset,depth=2,num_neighb=3)

G_data[0]

[0]
tensor([1862,  633,  633])


tensor([[   0,    0,    0, 1862, 1862, 1862,  633,  633,  633,  633,  633,  633],
        [1862,  633,  633,  926, 2582,  926, 1701,    0,    0, 1701,    0,    0]])

In [355]:
dataloader = DataLoader(G_data, batch_size=16,
                        shuffle=True, num_workers=4)

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Parameter
from torch_scatter import scatter_mean
from torch_geometric.utils import remove_self_loops, add_self_loops

from torch_geometric.nn.inits import uniform


class MyGCN(torch.nn.Module):

    def __init__(self, in_channels, out_channels, normalize=True, bias=True, number_relation=):
        super(SAGEConv, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.normalize = normalize
        self.weight = Parameter(torch.Tensor(self.in_channels, out_channels))

        if bias:
            self.bias = Parameter(torch.Tensor(out_channels))
        else:
            self.register_parameter('bias', None)

        self.reset_parameters()

    def reset_parameters(self):
        size = self.weight.size(0)
        uniform(size, self.weight)
        uniform(size, self.bias)


    def forward(self, x, edge_index):
        """"""
        x = x.unsqueeze(-1) if x.dim() == 1 else x
        row, col = edge_index

        x = torch.matmul(x, self.weight)
        out = scatter_mean(x[col], row, dim=0, dim_size=x.size(0))

        if self.bias is not None:
            out = out + self.bias

        if self.normalize:
            out = F.normalize(out, p=2, dim=-1)

        return out


    def __repr__(self):
        return '{}({}, {})'.format(self.__class__.__name__, self.in_channels,
                                   self.out_channels)

In [370]:
a = scatter_mean(torch.LongTensor([1,1,1]), torch.LongTensor([2,2,3]), dim=0, dim_size=4)

In [369]:
a

tensor([0, 0, 1, 1])